Install Needed Libraries 

In [ ]:
!pip install pytube
!pip install youtube_transcript_api

     |████████████████████████████████| 56 kB 2.1 MB/s 


Import Needed Libraries

In [ ]:
import json
import re
import os
import sys
import urllib.request
import requests
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from moviepy.editor import *
from bs4 import BeautifulSoup  

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3424256/45929032 bytes (7.5%)6979584/45929032 bytes (15.2%)10215424/45929032 bytes (22.2%)13639680/45929032 bytes (29.7%)16916480/45929032 bytes (36.8%)20406272/45929032 bytes (44.4%)23994368/45929032 bytes (52.2%)27295744/45929032 bytes (59.4%)30916608/45929032 bytes (67.3%)34398208/45929032 bytes (74.9%)37912576/45929032 bytes (82.5%)41320448/45929032 bytes (90.0%)

YouTube class to hold all the video data

In [ ]:
PATH = "/content/"
API_KEY = None

In [ ]:
# This class holds all the video data
class YoutubeStats:
    def __init__(self, url, id):
        #You can use the youtube data api v3 by uncommenting the following 3 lines and passing an API key to the class 
        #callUrl = f"https://www.googleapis.com/youtube/v3/videos?part=snippet&id={id}&key={API_KEY}"
        #self.respose = requests.get(callUrl)
        #self.data = json.loads(self.respose.text)
        try:
            self.captionData = YouTubeTranscriptApi.get_transcript(id)
        except:
            self.captionData = None

        self.url = url
        self.id = id
        self.title = self.getVideoTitle()

    def printData(self):
        print(self.data)

    def getURL(self):
        return self.url
    
    def getID(self):
        return self.id

    # Returns the video title
    def getVideoTitle(self):
        source = requests.get(self.url).text
        soup = BeautifulSoup(source,'html.parser')
        Title = soup.title.text
        print(Title)
        Title = Title[:-10] 
        return Title

    # Returns the video description as a string
    def getVideoDescription(self):
        return self.data["items"][0]["snippet"]["description"]

    # Downloads the video from youtube with 360p resolution
    # given the video URL and the name of the file
    def downloadVideo(self):
        path = PATH + 'videos'
        title = reNameTitle(self.title) + ".mp4"
        print("title ........... ", title)
        YouTube(self.url).streams.first().download(output_path=path, filename=title)

    # Coverts video from mp4 to mp3 given the name of
    # the file and the type of the output file eg: mp3, wav
    def convertVideoToSound(self, type="wav"):
        VideoPath = PATH + 'videos/'
        path = PATH + 'audio/'
        title = reNameTitle(self.title)
        video = VideoFileClip(VideoPath + title + '.mp4')
        video.audio.write_audiofile(path + title + '.' + type)

    # Returns the video transcript as a dictionary with times
    def getVideoTranscript(self):
        return self.captionData

    # Return the video transcript as a string
    def getVideoTranscriptString(self):
        transcript = ""
        addNewLine = False
        for caption in self.captionData:
            if addNewLine:
                transcript += str(caption["text"] + "\n")
                addNewLine = False
            else:
                transcript += str(caption["text"] + " ")
                addNewLine = True
        return transcript
    
    def writeToJsonFile(self):
        path = PATH + 'json_scripts'
        i = 0
        title = self.title
        transcript = self.getVideoTranscript()
        print("{0} Writing to json file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}.json", "w") as file:
            json.dump(transcript, file)
    def writeTranscriptFile(self):
        path = PATH + 'transcripts'
        i = 0
        title = self.title
        transcript = self.getVideoTranscriptString()
        print("{0} Writing to transcript file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}_transcript.txt", "w") as file:
            file.write(transcript)


Supporting Functions

In [ ]:
# This function create a file for every video
# and it assigns the file name to the title of
# that video. It then writes the title, description
# and the transcript to the file
def writeToVideoFile(youtubeStats):
    path = createDirectory('video_scripts')
    i = 0
    for stats in youtubeStats:
        title = stats.title
        #description = stats.getVideoDescription()
        transcript = stats.getVideoTranscriptString()
        print("{0} Writing to file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}.txt", "w") as file:
            file.write("Title\n\n" + title)
            file.write(
                "\n--------------------------------------------------------------------\n")
            #file.write("\nDescription\n\n" + description)
            #file.write(
            #    "\n\n--------------------------------------------------------------------\n")
            file.write("\nTranscript\n\n" + transcript)
            file.write(
                "\n--------------------------------------------------------------------\n")

In [ ]:
def writeAllToJsonFile(youtubeStats):
    path = createDirectory('Json_scripts')
    i = 0
    for stats in youtubeStats:
        title = stats.title
        transcript = stats.getVideoTranscript()
        print("{0} Writing to json file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}.json", "w") as file:
            json.dump(transcript, file)

In [ ]:
#download all the videos in the input file
def downloadAllVideos(youtubeStats):
    createDirectory('videos')   
    i = 0
    for stats in youtubeStats:
        print("{0} Downloading...".format(i+1))
        i+=1
        stats.downloadVideo(stats.getURL(), stats.title) 

In [ ]:
def convertAllVideosToSound(youtubeStats):
    createDirectory('audio')
    for stats in youtubeStats:
        stats.convertVideoToSound(stats.title, 'wav')


In [ ]:
# Helper function to rename the file
def reNameTitle(title):
    title = re.sub('[\W_]+', "_", title)
    return title.lower()

In [ ]:
# Helper function to create a directory
def createDirectory(name):
    path = PATH + name
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

In [ ]:
def converYoutubeLinkToId(link):
  return link.split("=", 1)[1].strip()

In [ ]:
# Reads a list of URLs from a given file
# and parses them to video IDs.
# Returns a list of URLs and video IDs
def readFile():
    videoIds = []
    urls = []
    fileName = "testLinks.txt"
    path = PATH
    with open(path+fileName, "r") as file:
        for line in file:
            videoIds.append(line.split("=", 1)[1].strip())
            urls.append(line.strip())
    return videoIds, urls

Main

In [ ]:
def main():
    #Reading all the urls from the file
    videoIds, urls = readFile()
    createDirectory('json_scripts')
    createDirectory('audio')
    createDirectory('videos')
    createDirectory('transcripts')
    
    for i, id in enumerate(videoIds):
        temp=YoutubeStats(urls[i], id)
        if(temp.captionData is None):
            continue
        #temp.downloadVideo()
        #temp.convertVideoToSound()
        #temp.writeToJsonFile()
        temp.writeTranscriptFile()
       # temp.writeToVideoFile()

In [ ]:
#https://www.youtube.com/watch?v=qfXMkt67mqM
videoURL = input("Enter the Youtube video link: ")
videoId = converYoutubeLinkToId(videoURL)
youtubeStats = YoutubeStats(videoURL, videoId)
print(youtubeStats.getVideoTranscriptString())

In [ ]:
# !rm -r dir1 audio
# !rm -r dir1 videos
# !rm -r dir1 Json_scripts  
#!rm -r dir1 transcripts  